### Project Description: Twitter US Airline Sentiment¶

A sentiment analysis job about the problems of each major U.S. airline. Twitter data was scraped from February of 2015 and contributors were asked to first classify positive, negative, and neutral tweets, followed by categorizing negative reasons (such as "late flight" or "rude service").

**Dataset:**
The project is from a dataset from Kaggle.

Link to the Kaggle project site: https://www.kaggle.com/crowdflower/twitter-airline-sentiment The dataset has to be downloaded from the above Kaggle website. The dataset has the following columns:

• tweet_id • airline_sentiment • airline_sentiment_confidence • negativereason • negativereason_confidence • airline • airline_sentiment_gold • name • negativereason_gold • retweet_count • text • tweet_coord • tweet_created • tweet_location • user_timezone

Objective:

To implement the techniques learnt as a part of the course.

**Learning Outcomes:¶**

• Basic understanding of text pre-processing. • What to do after text pre-processing: o Bag of words o Tf-idf • Build the classification model. • Evaluate the Model.

### Steps and tasks:
1. Import the libraries, load dataset, print shape of data, data description. (5 Marks)
2. Understand of data-columns: (5 Marks)
    a. Drop all other columns except “text” and “airline_sentiment”.
    b. Check the shape of data.
    c. Print first 5 rows of data.
3. Text pre-processing: Data preparation. (20 Marks)
    a. Html tag removal.
    b. Tokenization.
    c. Remove the numbers.
    d. Removal of Special Characters and Punctuations.
    e. Conversion to lowercase.
    f. Lemmatize or stemming.
    g. Join the words in the list to convert back to text string in the dataframe. (So that each row contains the data in text format.)
    h. Print first 5 rows of data after pre-processing.
4. Vectorization: (10 Marks)
    a. Use CountVectorizer.
    b. Use TfidfVectorizer.
5. Fit and evaluate model using both type of vectorization. (6+6 Marks)
6. Summarize your understanding of the application of Various Pre-processing and Vectorization and performance of your model on this dataset. (8 Marks)

In [6]:
!pip install nltk

In [7]:
!pip install contractions

     |████████████████████████████████| 245kB 8.2MB/s 
     |████████████████████████████████| 317kB 13.6MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.0-cp36-cp36m-linux_x86_64.whl size=81696 sha256=506b1a8da46036c56107f89ca40affdf4022f5d0b3ab4d328e90789dcc54dd0e
  Stored in directory: /root/.cache/pip/wheels/0a/90/61/87a55f5b459792fbb2b7ba6b31721b06ff5cf6bde541b40994
Successfully built pyahocorasick


In [8]:
!pip install fastText

     |████████████████████████████████| 71kB 4.3MB/s 
  Created wheel for fastText: filename=fasttext-0.9.2-cp36-cp36m-linux_x86_64.whl size=3040203 sha256=e3d2d53aa8e51c1406c2cbff51aa749ef425bf6aa51b06422ef067239f48dfe7
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built fastText


In [9]:
# from google.colab import drive 
# drive.mount ('/content/drive', force_remount=True)
# path = '/content/drive/MyDrive/Colab Notebooks/Data_002_NLP/'

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
!ls '/content/drive/MyDrive/Colab Notebooks/Data_002_NLP/'

Tweets.csv


In [12]:
# Import necessary libraries.

import pandas as pd
import numpy as np
import csv

import re, string, unicodedata
import nltk                                   # Natural language processing tool-kit

import contractions
import string
import fasttext

from bs4 import BeautifulSoup                 # Beautiful soup is a parsing library that can use different parsers.
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords, wordnet    # Stopwords, and wordnet corpus
from nltk.stem import LancasterStemmer, WordNetLemmatizer

In [13]:
# Open and read objects Airline Tweeter File

AirlineTweets_F01 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data_002_NLP/Tweets.csv', sep='\t')

In [14]:
AirlineTweets_F01.shape

(14813, 1)

In [15]:
AirlineTweets_F01.values

array([['570306133677760513,neutral,1.0,,,Virgin America,,cairdin,,0,@VirginAmerica What @dhepburn said.,,2015-02-24 11:35:52 -0800,,Eastern Time (US & Canada)'],
       ["570301130888122368,positive,0.3486,,0.0,Virgin America,,jnardino,,0,@VirginAmerica plus you've added commercials to the experience... tacky.,,2015-02-24 11:15:59 -0800,,Pacific Time (US & Canada)"],
       ["570301083672813571,neutral,0.6837,,,Virgin America,,yvonnalynn,,0,@VirginAmerica I didn't today... Must mean I need to take another trip!,,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)"],
       ...,
       ['569587242672398336,neutral,1.0,,,American,,sanyabun,,0,@AmericanAir Please bring American Airlines to #BlackBerry10,,2015-02-22 11:59:15 -0800,"Nigeria,lagos",'],
       ['569587188687634433,negative,1.0,Customer Service Issue,0.6659,American,,SraJackson,,0,"@AmericanAir you have my money, you change my flight, and don\'t answer your phones! Any other suggestions so I can make my commitment?

In [16]:
print(AirlineTweets_F01.describe())

       tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
count                                               14813                                                                                                                                                                                
unique                                              14774                                                                                                                                                                                
top     Keep up the good work folks, this is where Ces...                                                                                                                                                                                
freq                                                    2       

In [17]:
# Display Contents/details of Tweeter file - Display columns and data in csv file

with open('/content/drive/MyDrive/Colab Notebooks/Data_002_NLP/Tweets.csv', 'r') as csv_file:
          AirlineTweets_F01 = csv.reader(csv_file)
          
          for line in AirlineTweets_F01:
              print(line)

Streaming output truncated to the last 5000 lines.
['568777943104802817', 'negative', '0.6632', 'Bad Flight', '0.3474', 'Southwest', '', 'dealWHITit', '', '0', "@SouthwestAir it's too damn cold to be sitting on this runway with no heat!!!!!", '', '2015-02-20 06:23:23 -0800', '', 'Indiana (East)']
['568776728333541376', 'neutral', '0.3469', '', '0.0', 'Southwest', '', 'Milwaukette', '', '0', '@SouthwestAir Can you please follow so I can DM? Thx!', '', '2015-02-20 06:18:34 -0800', 'Milwaukee, WI', 'Atlantic Time (Canada)']
['568774192029523969', 'neutral', '1.0', '', '', 'Southwest', '', 'tracyannmanning', '', '0', '@SouthwestAir sent DM', '', '2015-02-20 06:08:29 -0800', 'Suwanee, GA', 'Pacific Time (US & Canada)']
['568773575705899008', 'positive', '1.0', '', '', 'Southwest', '', 'tradingtrainer', '', '0', "@SouthwestAir... I love you. Air travel doesn't get easier.", '', '2015-02-20 06:06:02 -0800', 'Colorado', 'Mountain Time (US & Canada)']
['568772912557133824', 'positive', '1.0', '

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [18]:
# Display CSV file contents by Column (comma separated)

with open('/content/drive/MyDrive/Colab Notebooks/Data_002_NLP/Tweets.csv', 'r') as csv_file:
          AirlineTweets_F01 = csv.reader(csv_file)
          
          for line in AirlineTweets_F01:
              print(line)

Streaming output truncated to the last 5000 lines.
['570261514340474880', 'negative', '1.0', 'Customer Service Issue', '0.6382', 'Southwest', '', 'island_girl4321', '', '0', '@SouthwestAir  on hold with customer service; you may be quicker.  I rebooked trip for a lower fare.  Where did the difference in $ go?', '', '2015-02-24 08:38:34 -0800', '', '']
['570261349172973569', 'neutral', '1.0', '', '', 'Southwest', '', 'laurabogart14', '', '0', '@SouthwestAir That was Chantilly, Paris, France.', '', '2015-02-24 08:37:55 -0800', 'Raleigh, NC', 'Eastern Time (US & Canada)']
['570261289420898305', 'negative', '1.0', 'Customer Service Issue', '0.6526', 'Southwest', '', 'ChefBillyParisi', '', '0', '@SouthwestAir and thx for not responding', '', '2015-02-24 08:37:40 -0800', 'Chicago', 'Mountain Time (US & Canada)']
['570261225520689152', 'negative', '1.0', 'Customer Service Issue', '0.6224', 'Southwest', '', 'JBadnastythug', '', '0', '@SouthwestAir I booked a flight on my phone and then never g

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Read and Display Airline Tweet file with column Headers

In [19]:
import pandas
AirlineTweets_F02 = pandas.read_csv('/content/drive/MyDrive/Colab Notebooks/Data_002_NLP/Tweets.csv', sep=',')
print(AirlineTweets_F02)

                 tweet_id  ...               user_timezone
0      570306133677760513  ...  Eastern Time (US & Canada)
1      570301130888122368  ...  Pacific Time (US & Canada)
2      570301083672813571  ...  Central Time (US & Canada)
3      570301031407624196  ...  Pacific Time (US & Canada)
4      570300817074462722  ...  Pacific Time (US & Canada)
...                   ...  ...                         ...
14635  569587686496825344  ...                         NaN
14636  569587371693355008  ...                         NaN
14637  569587242672398336  ...                         NaN
14638  569587188687634433  ...  Eastern Time (US & Canada)
14639  569587140490866689  ...                         NaN

[14640 rows x 15 columns]


In [20]:
AirlineTweets_F02.shape

(14640, 15)

In [21]:
AirlineTweets_F02.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [22]:
AirlineTweets_F02.describe()

,tweet_id,airline_sentiment_confidence,negativereason_confidence,retweet_count
count,1.464000e+04,14640.000000,10522.000000,14640.000000
mean,5.692184e+17,0.900169,0.638298,0.082650
std,7.791112e+14,0.162830,0.330440,0.745778
min,5.675883e+17,0.335000,0.000000,0.000000
25%,5.685592e+17,0.692300,0.360600,0.000000
50%,5.694779e+17,1.000000,0.670600,0.000000
75%,5.698905e+17,1.000000,1.000000,0.000000
max,5.703106e+17,1.000000,1.000000,44.000000


### 2. Understand of data-columns: (5 Marks)

2a. Drop all other columns except “text” and “airline_sentiment”.

In [28]:
AirlineTweets_F03 = AirlineTweets_F02.loc[:14813, ['text','airline_sentiment']]

In [29]:
AirlineTweets_F03.shape

(14640, 2)

2c. Print of first 5 rows of data

In [30]:
AirlineTweets_F03.head()

,text,airline_sentiment
0,@VirginAmerica What @dhepburn said.,neutral
1,@VirginAmerica plus you've added commercials t...,positive
2,@VirginAmerica I didn't today... Must mean I n...,neutral
3,@VirginAmerica it's really aggressive to blast...,negative
4,@VirginAmerica and it's a really big bad thing...,negative


In [38]:
# First row of data.
pd.set_option('display.max_colwidth', None) # It will enable the entire row visible with truncation of the text. (We can see full text.)
AirlineTweets_F03.loc[[0]]

,text,airline_sentiment
0,@VirginAmerica What @dhepburn said.,neutral


### 3. Text pre-processing: Data preparation. (20 Marks)

3a. Html tag removal.

In [40]:
# Removal of the http link using Regular Expression.
for i, row in AirlineTweets_F03.iterrows():
    clean_text = re.sub(r"http\S+", "", AirlineTweets_F03.at[i, 'text'])
    AirlineTweets_F03.at[i,'text'] = clean_text
AirlineTweets_F03.head()

,text,airline_sentiment
0,@VirginAmerica What @dhepburn said.,neutral
1,@VirginAmerica plus you have added commercials to the experience... tacky.,positive
2,@VirginAmerica I did not today... Must mean I need to take another trip!,neutral
3,"@VirginAmerica it is really aggressive to blast obnoxious ""entertainment"" in your guests' faces &amp; they have little recourse",negative
4,@VirginAmerica and it is a really big bad thing about it,negative


3b. Tokenization of the Text.

In [55]:
#tokenization
# import nltk
from nltk.tokenize.toktok import ToktokTokenizer

for i, row in AirlineTweets_F03.iterrows():
  tokenizer=ToktokTokenizer()
  text01 = AirlineTweets_F03
  # text= "The , and , if are stopwords, computer is not"
  tokens=tokenizer.tokenize(text01)
AirlineTweets_F03.head()
# print(tokens)

,text,airline_sentiment
0,@VirginAmerica What @dhepburn said.,neutral
1,@VirginAmerica plus you have added commercials to the experience... tacky.,positive
2,@VirginAmerica I did not today... Must mean I need to take another trip!,neutral
3,"@VirginAmerica it is really aggressive to blast obnoxious ""entertainment"" in your guests' faces &amp; they have little recourse",negative
4,@VirginAmerica and it is a really big bad thing about it,negative


In [56]:
print(tokens)

['text', 'airline_sentiment', '0', '@VirginAmerica', 'What', '@dhepburn', 'said.', 'neutral', '1', '@VirginAmerica', 'plus', 'you', 'have', 'added', 'commercials', 'to', 'the', 'experience', '...', 'tacky.', 'positive', '2', '@VirginAmerica', 'I', 'did', 'not', 'today', '...', 'Must', 'mean', 'I', 'need', 'to', 'take', 'another', 'trip', '!', 'neutral', '3', '@VirginAmerica', 'it', 'is', 'really', 'aggressive', 'to', 'blast', 'obnoxious', '"', 'entertainment', '"', 'in', 'your', 'guests', "'", 'faces', '&amp', ';', 'they', 'have', 'little', 'recourse', 'negative', '4', '@VirginAmerica', 'and', 'it', 'is', 'a', 'really', 'big', 'bad', 'thing', 'about', 'it', 'negative', '...', '...', '...', '14635', '@AmericanAir', 'thank', 'you', 'we', 'got', 'on', 'a', 'different', 'flight', 'to', 'Chicago.', 'positive', '14636', '@AmericanAir', 'leaving', 'over', '20', 'minutes', 'Late', 'Flight.', 'No', 'warnings', 'or', 'communication', 'until', 'we', 'were', '15', 'minutes', 'Late', 'Flight.', 'th

In [46]:
#tokenization
# import nltk

# from nltk.tokenize.toktok import ToktokTokenizer

# tokenizer=ToktokTokenizer()
# text01 = AirlineTweets_F03
# tokens=tokenizer.tokenize(text01)
# print(tokens)

3c. Remove numbers from the Text

In [86]:
# Removing Numbers from the Text
# import re

def remove_special_characters(AirlineTweets_F03, remove_digits=False):
    pattern01 = r'[^a-zA-z\s]' if not remove_digits else r'[^a-zA-z\s]'
    AirlineTweets_F03 = re.sub(pattern01, '', text)
    return AirlineTweets_F03

remove_special_characters(AirlineTweets_F03, 
                          remove_digits=True)

'VirginAmerica What dhepburn said'

3d. Removal of Special Characters and Punctuations.

In [87]:
def remove_special_characters(AirlineTweets_F03, remove_digits=False):
    pattern01 = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    AirlineTweets_F03 = re.sub(pattern01, '', text)
    return AirlineTweets_F03

remove_special_characters(AirlineTweets_F03, 
                          remove_digits=True)

'VirginAmerica What dhepburn said'

In [88]:
# def remove_non_ascii(AirlineTweets_F03words):
#    """Remove non-ASCII characters from list of tokenized words"""
#    AirlineTweets_F04 = []                        # Create empty list to store pre-processed words.
#    for word in AirlineTweets_F03:
#        AirlineTweets_F03 = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
#        AirlineTweets_F04.append(AirlineTweets_F03)        # Append processed words to new list.
#    return AirlineTweets_F04

3e. Conversion to lowercase.

In [96]:
def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []                        # Create empty list to store pre-processed words.
    for word in words:
        new_word = word.lower()           # Converting to lowercase
        new_words.append(new_word)        # Append processed words to new list.
    return new_words

In [97]:
# def to_lowercase(AirlineTweets_F04):
#    """Convert all characters to lowercase from list of tokenized words"""
#    new_AirlineTweets_F04 = []                        # Create empty list to store pre-processed words.
#    for word in new_AirlineTweets_F04:
#        new_word = word.lower()           # Converting to lowercase
#        new_AirlineTweets_F04.append(new_word)        # Append processed words to new list.
#    return new_AirlineTweets_F04

3f. Lemmatize and stemming.

In [98]:
# Stemming
def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []                            # Create empty list to store pre-processed words.
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)                # Append processed words to new list.
    return stems

In [99]:
# Lemmitization
def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []                           # Create empty list to store pre-processed words.
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)              # Append processed words to new list.
    return lemmas

In [100]:
# def lemmatize_verbs(words):
#    """Lemmatize verbs in list of tokenized words"""
#    lemmatizer = WordNetLemmatizer()
#    lemmas = []                           # Create empty list to store pre-processed words.
#    for word in words:
#        lemma = lemmatizer.lemmatize(word, pos='v')
#        lemmas.append(lemma)              # Append processed words to new list.
#    return lemmas

Normalize function to processes all the steps together.¶

In [101]:
# def normalize(words):
#    AirlineTweets_F04 = remove_non_ascii(words)
#    new_AirlineTweets_F0 = to_lowercase(words)
#    words = remove_punctuation(words)
#    words = remove_stopwords(words)
#    return words

In [105]:
def normalize(words):
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = remove_stopwords(words)
    return words

In [107]:
# Iterate the normalize funtion over whole data.
for i, row in AirlineTweets_F03.iterrows():
    words = AirlineTweets_F03.at[i, 'text']
#    words = normalize(words)
    AirlineTweets_F03.at[i,'text'] = words
AirlineTweets_F03.head()

,text,airline_sentiment
0,@VirginAmerica What @dhepburn said.,neutral
1,@VirginAmerica plus you have added commercials to the experience... tacky.,positive
2,@VirginAmerica I did not today... Must mean I need to take another trip!,neutral
3,"@VirginAmerica it is really aggressive to blast obnoxious ""entertainment"" in your guests' faces &amp; they have little recourse",negative
4,@VirginAmerica and it is a really big bad thing about it,negative


In [108]:
def stem_and_lemmatize(words):
    stems = stem_words(words)
    lemmas = lemmatize_verbs(words)
    return stems, lemmas

In [110]:
# import nltk
nltk.download('wordnet')

AirlineTweets_F03['lemma'] = ''
AirlineTweets_F03['stem'] = ''

for i, row in AirlineTweets_F03.iterrows():
    words = AirlineTweets_F03.at[i, 'text']
    stems, lemmas = stem_and_lemmatize(words)
    AirlineTweets_F03.at[i,'stem'] = stems
    AirlineTweets_F03.at[i, 'lemma'] = lemmas
AirlineTweets_F03.head()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


,text,airline_sentiment,lemma,stem
0,@VirginAmerica What @dhepburn said.,neutral,"[@, V, i, r, g, i, n, A, m, e, r, i, c, a, , W, h, a, t, , @, d, h, e, p, b, u, r, n, , s, a, i, d, .]","[@, v, i, r, g, i, n, a, m, e, r, i, c, a, , w, h, a, t, , @, d, h, e, p, b, u, r, n, , s, a, i, d, .]"
1,@VirginAmerica plus you have added commercials to the experience... tacky.,positive,"[@, V, i, r, g, i, n, A, m, e, r, i, c, a, , p, l, u, s, , y, o, u, , h, a, v, e, , a, d, d, e, d, , c, o, m, m, e, r, c, i, a, l, s, , t, o, , t, h, e, , e, x, p, e, r, i, e, n, c, e, ., ., ., , t, a, c, k, y, .]","[@, v, i, r, g, i, n, a, m, e, r, i, c, a, , p, l, u, s, , y, o, u, , h, a, v, e, , a, d, d, e, d, , c, o, m, m, e, r, c, i, a, l, s, , t, o, , t, h, e, , e, x, p, e, r, i, e, n, c, e, ., ., ., , t, a, c, k, y, .]"
2,@VirginAmerica I did not today... Must mean I need to take another trip!,neutral,"[@, V, i, r, g, i, n, A, m, e, r, i, c, a, , I, , d, i, d, , n, o, t, , t, o, d, a, y, ., ., ., , M, u, s, t, , m, e, a, n, , I, , n, e, e, d, , t, o, , t, a, k, e, , a, n, o, t, h, e, r, , t, r, i, p, !]","[@, v, i, r, g, i, n, a, m, e, r, i, c, a, , i, , d, i, d, , n, o, t, , t, o, d, a, y, ., ., ., , m, u, s, t, , m, e, a, n, , i, , n, e, e, d, , t, o, , t, a, k, e, , a, n, o, t, h, e, r, , t, r, i, p, !]"
3,"@VirginAmerica it is really aggressive to blast obnoxious ""entertainment"" in your guests' faces &amp; they have little recourse",negative,"[@, V, i, r, g, i, n, A, m, e, r, i, c, a, , i, t, , i, s, , r, e, a, l, l, y, , a, g, g, r, e, s, s, i, v, e, , t, o, , b, l, a, s, t, , o, b, n, o, x, i, o, u, s, , "", e, n, t, e, r, t, a, i, n, m, e, n, t, "", , i, n, , y, o, u, r, , g, u, e, s, t, s, ', , f, a, c, e, s, , &, a, m, p, ...]","[@, v, i, r, g, i, n, a, m, e, r, i, c, a, , i, t, , i, s, , r, e, a, l, l, y, , a, g, g, r, e, s, s, i, v, e, , t, o, , b, l, a, s, t, , o, b, n, o, x, i, o, u, s, , "", e, n, t, e, r, t, a, i, n, m, e, n, t, "", , i, n, , y, o, u, r, , g, u, e, s, t, s, ', , f, a, c, e, s, , &, a, m, p, ...]"
4,@VirginAmerica and it is a really big bad thing about it,negative,"[@, V, i, r, g, i, n, A, m, e, r, i, c, a, , a, n, d, , i, t, , i, s, , a, , r, e, a, l, l, y, , b, i, g, , b, a, d, , t, h, i, n, g, , a, b, o, u, t, , i, t]","[@, v, i, r, g, i, n, a, m, e, r, i, c, a, , a, n, d, , i, t, , i, s, , a, , r, e, a, l, l, y, , b, i, g, , b, a, d, , t, h, i, n, g, , a, b, o, u, t, , i, t]"


### 4. Vectorization: (10 Marks)
a. Use CountVectorizer.
b. Use TfidfVectorizer.

In [111]:
# Vectorization (Convert text data to numbers).
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_features=1000)                # Keep only 1000 features as number of features will increase the processing time.
data_features = vectorizer.fit_transform(AirlineTweets_F03['text'])

data_features = data_features.toarray()                        # Convert the data features to array.

In [112]:
data_features.shape

(14640, 1000)

In [116]:
labels = AirlineTweets_F03['airline_sentiment']
labels = labels.astype('string')

In [117]:
# Split data into training and testing set.

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data_features, labels, test_size=0.3, random_state=42)

In [118]:
# Using Random Forest to build model for the classification of reviews.
# Also calculating the cross validation score.

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

forest = RandomForestClassifier(n_estimators=10, n_jobs=4)

forest = forest.fit(X_train, y_train)

print(forest)

print(np.mean(cross_val_score(forest, data_features, labels, cv=10)))

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=4,
                       oob_score=False, random_state=None, verbose=0,
                       warm_start=False)
0.710997267759563


In [119]:
# Predict the result for test data using the model built above.

result = forest.predict(X_test)

In [128]:
# Print and plot Confusion matirx to get an idea of how the distribution of the prediction is, among all the classes.

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

conf_mat = confusion_matrix(y_test, result)

print(conf_mat)


df_cm = pd.DataFrame(conf_mat, index = [i for i in "14640"],
                  columns = [i for i in "14640"])
plt.figure(figsize = (10,7))
sns.heatmap(df_cm, annot=True, fmt='g')

[[2609  150   55]
 [ 439  385   60]
 [ 257  114  323]]


ValueError: ignored

In [126]:
# Using TfidfVectorizer to convert text data to numbers.

from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=1000)
data_features = vectorizer.fit_transform(AirlineTweets_F03['text'])

data_features = data_features.toarray()

data_features.shape

(14640, 1000)

In [122]:
# Using Random Forest to build model for the classification of reviews.
# Also calculating the cross validation score.

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

import numpy as np

forest = RandomForestClassifier(n_estimators=10, n_jobs=4)

forest = forest.fit(X_train, y_train)

print(forest)

print(np.mean(cross_val_score(forest, data_features, labels, cv=10)))

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=4,
                       oob_score=False, random_state=None, verbose=0,
                       warm_start=False)
0.7161885245901639


In [123]:
result = forest.predict(X_test)

6. Summarize your understanding of the application of Various Pre-processing and Vectorization and performance of your model on this dataset. (8 Marks)

Summary
1. Sentiment values of 'neutral', 'positive'and 'negative'
2. Text Preprocessing helps improve accuracy of the text analysis
3. Pre-processing of the text data requires installing and importing various NLP linbraries
